In [ ]:
def single_iv(data,var,target,eps):
    ## The number of every class corresponding to a attribute's value
    count_attr = pd.crosstab(data[var],data[target])+eps
    
    ## The number of every class
    count_class = data[target].value_counts()+eps

    result = count_attr/count_class
    iv = (result[1]-result[0])*np.log(result[1]/result[0])
    iv = iv.sum()

    return round(iv,6)

def multi_iv(data,var1,var2,target,eps):
    
    count_multi_attr = pd.crosstab([data[var1],data[var2]],data[target])+eps
    count_class = data[target].value_counts()+eps
    
    result = count_multi_attr/count_class
    iv = (result[1]-result[0])*np.log(result[1]/result[0])
    iv = iv.sum()
    return round(iv,6)

## 信息增益和信息增益比
def IG_IGR(data,var,target,eps):
    ## H(Y)
    entripy_class = (data[target].value_counts()+eps)/len(data)
    entripy_class = (-entripy_class*np.log2(entripy_class)).sum()
    
    count_attr = pd.crosstab(data[var],data[target],margins=True)+eps
    
    ## H(A)
    prob_attr = np.array(count_attr['All']/len(data))
    ## The probability of feature's each value
    entripy_attr = (-prob_attr*np.log2(prob_attr)).sum()
    ## The entripy of attribute
    
    ## H(Y|A=a)
    for each in count_attr.columns:
        count_attr[each] = count_attr[each]/count_attr['All']
    count_attr = - count_attr*np.log2(count_attr)
    entripy_each = np.sum(count_attr,axis=1)
    entripy_each = np.array(entripy_each)
    
    ## H(Y|A)
    entripy = np.sum(prob_attr*entripy_each)
    ## The entripy of target respect to attribute
    
    ## Information Gain
    IG = entripy_class - entripy
    ## Information Gain Ratio
    IGR = IG/entripy_attr
    
    return round(IG,6),round(IGR,6)

def multi_IG_IGR(data,var1,var2,target,eps):
    ## H(Y)
    entripy_class = (data[target].value_counts()+eps)/len(data)
    entripy_class = (-entripy_class*np.log2(entripy_class)).sum()
    
    count_attr = pd.crosstab([data[var1],data[var2]],data[target],margins=True)+eps
    
    ## H(A1,A2)
    prob_attr = np.array(count_attr['All']/len(data))
    entripy_attr = (-prob_attr*np.log2(prob_attr)).sum()
    
    ## H(Y|A1=a,A2=b)
    for each in count_attr.columns:
        count_attr[each] = count_attr[each]/count_attr['All']
    count_attr = - count_attr*np.log2(count_attr)
    entripy_each = np.sum(count_attr,axis=1)
    entripy_each = np.array(entripy_each)
    
    ## H(Y|A1,A2)
    ## The entripy of target respect to attribute
    entripy = prob_attr*entripy_each
    entripy = np.sum(entripy[:-1])

    ## Information Gain
    IG = entripy_class - entripy
    ## Information Gain Ratio
    IGR = IG/entripy_attr
    
    return round(IG,6),round(IGR,6)


equip = pd.read_csv(path+"Archive(1)/equip_history.csv",sep=',')
attribute = [each for each in equip.columns[1:-1] if 'R' not in each ]
re_colunms = ['Atrribute_1','Atrribute_2','A1_IV','A1_IG','A1_GR','A2_IV','A2_IG','A2_GR','Inter_IV','Inter_IG','Inter_GR',
              'Inter_IVI','Inter_IVL','Inter_IGI','Inter_IGL','Inter_GRI','Inter_GRL']
result = []
for i in range(len(attribute)):
    iv = single_iv(equip,attribute[i],'Y',1e-6)
    ig,gr = IG_IGR(equip,attribute[i],'Y',1e-6)
    for j in range(i+1,len(attribute)):
        row = [0]*len(re_colunms)
        row[0] = attribute[i]
        row[1] = attribute[j]
        row[2] = iv
        row[3] = ig
        row[4] = gr
        
        row[5] = single_iv(equip,attribute[j],'Y',1e-6)
        row[6],row[7] = IG_IGR(equip,attribute[j],'Y',1e-6)
        
        row[8] = multi_iv(equip,attribute[i],attribute[j],'Y',1e-6)
        row[9],row[10] = multi_IG_IGR(equip,attribute[i],attribute[j],'Y',1e-6)
        
        row[11] = round(row[8] - max(row[2],row[5]),6)
        row[12] = round(row[8]/max(row[2],row[5]),3)
        row[13] = round(row[9] - max(row[3],row[6]),6)
        row[14] = round(row[9]/max(row[3],row[6]),3)
        row[15] = round(row[10] - max(row[4],row[7]),6)
        row[16] = round(row[10]/max(row[4],row[7]),3)
        result.append(row)
print("Ending")

result = pd.DataFrame(result,columns=re_colunms)
result.to_csv(path+"Archive(1)/interaction.csv")